In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torchvision.transforms as transforms

from torch.autograd import Variable #Variable is depreciated but can still be used. Autograd automatically supports Tensors with requires_grad set to True
from torchvision.utils import save_image


# Now we load the data

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.3, 0.35))])

#Compose is used to chain various transformations together

trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.4914, 0.4822, 0.4466), (0.247, 0.243, 0.261))])

trainset = tv.datasets.CIFAR10(root='./data',  train=True,download=True, transform=transform)

dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False, num_workers=4)

testset = tv.datasets.CIFAR10(root='./data', train=False, download=True, transform=trainTransform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  # 10 classes

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

#it = iter(dataloader)
#image = it.next()
#print(image[1])

# Make the model

class Autoencoder(nn.Module):
  
  def __init__(self):
    super(Autoencoder, self).__init__()

    self.encoder = nn.Sequential(nn.Conv2d(3, 6, kernel_size=5),nn.MaxPool2d(5,2,return_indices = True),nn.Conv2d(6,16,kernel_size=5),nn.ReLU(True))
    
    self.decoder = nn.Sequential(             
    nn.ConvTranspose2d(16,6,kernel_size=5),
    nn.MaxUnpool2d(5,2),
    nn.ConvTranspose2d(6,3,kernel_size=5),
    nn.ReLU(True),
    nn.Sigmoid())

  def forward(self,x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x

#defining some parameters

num_epochs = 5
batch_size = 128

model = Autoencoder().cpu()

distance = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cpu()
        
        #forward propogation
        
        output = model(img)
        loss = distance(output, img) #distance between original image and compressed image
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))

Files already downloaded and verified
Files already downloaded and verified
